In [8]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 2.9 MB/s eta 0:00:00


In [9]:
pip install requests

In [10]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import itertools
from geopy.distance import geodesic
import time
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import requests
import json

# Configure settings
ox.settings.use_cache = True
ox.settings.log_console = True

In [11]:
# --- Cell 2: Raw data (from JSON) ---
data = {
    "hospital": {
        "name": "Central Hospital",
        "latitude": 29.99512653425452,
        "longitude": 31.68462840171934,
        "type": "destination"
    },
    "patients": [
        {"id": "DT", "name": "Patient DT", "latitude": 30.000417586266437, "longitude": 31.73960813272627},
        {"id": "GR", "name": "Patient GR", "latitude": 30.011344405285193, "longitude": 31.747827362371993},
        {"id": "R2", "name": "Patient R2", "latitude": 30.030388325206854, "longitude": 31.669231198639675},
        {"id": "R3_2", "name": "Patient R3_2", "latitude": 30.030940768851426, "longitude": 31.688371339937028},
        {"id": "IT", "name": "Patient IT", "latitude": 30.01285635906825, "longitude": 31.693811715848444}
    ]
}

In [12]:
# --- Cell 3: Convert to DataFrame ---
hospital_df = pd.DataFrame([data["hospital"]])
patients_df = pd.DataFrame(data["patients"])
patients_df["type"] = "patient"
locations_df = pd.concat([hospital_df, patients_df], ignore_index=True)
locations_df.drop_duplicates(subset=["latitude", "longitude"], inplace=True)
locations_df.reset_index(drop=True, inplace=True)

print("Locations DataFrame:")
locations_df

Locations DataFrame:


,name,latitude,longitude,type,id
0,Central Hospital,29.995127,31.684628,destination,NaN
1,Patient DT,30.000418,31.739608,patient,DT
2,Patient GR,30.011344,31.747827,patient,GR
3,Patient R2,30.030388,31.669231,patient,R2
4,Patient R3_2,30.030941,31.688371,patient,R3_2
5,Patient IT,30.012856,31.693812,patient,IT


In [13]:
def calculate_osrm_distances(locations_df):
    """Use OSRM API for accurate road distance calculations"""
    coordinates = []
    location_names = []

    # Add hospital first
    hospital = locations_df[locations_df['type'] == 'destination'].iloc[0]
    coordinates.append(f"{hospital['longitude']},{hospital['latitude']}")
    location_names.append(hospital['name'])

    # Add patients
    for idx, row in locations_df[locations_df['type'] == 'patient'].iterrows():
        coordinates.append(f"{row['longitude']},{row['latitude']}")
        location_names.append(row['name'])

    coords_str = ';'.join(coordinates)
    url = f"http://router.project-osrm.org/table/v1/driving/{coords_str}?annotations=distance"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if 'distances' in data:
            distance_matrix = {}

            for i, loc1 in enumerate(location_names):
                distance_matrix[loc1] = {}
                for j, loc2 in enumerate(location_names):
                    distance_meters = data['distances'][i][j]
                    distance_km = distance_meters / 1000 if distance_meters is not None else float('inf')
                    distance_matrix[loc1][loc2] = distance_km

            return distance_matrix
        else:
            print("OSRM response format error")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None


In [14]:
# Calculate distances using OSRM
print("Calculating real road distances using OSRM API...")
distance_matrix = calculate_osrm_distances(locations_df)

if distance_matrix:
    print("Real road distance matrix (km):")
    distance_df = pd.DataFrame(distance_matrix)
    print(distance_df.round(2))
else:
    print("Falling back to haversine distance...")
    # Fallback to haversine distance
    distance_matrix = {}
    for i, row1 in locations_df.iterrows():
        distance_matrix[row1['name']] = {}
        for j, row2 in locations_df.iterrows():
            if i == j:
                distance_matrix[row1['name']][row2['name']] = 0
            else:
                dist = geodesic((row1['latitude'], row1['longitude']),
                               (row2['latitude'], row2['longitude'])).km
                distance_matrix[row1['name']][row2['name']] = dist
    print("Haversine distance matrix (km):")
    distance_df = pd.DataFrame(distance_matrix)
    print(distance_df.round(2))

Calculating real road distances using OSRM API...
Real road distance matrix (km):
                  Central Hospital  Patient DT  Patient GR  Patient R2  \
Central Hospital              0.00       14.19       17.78       11.86   
Patient DT                    8.63        0.00        7.75       19.67   
Patient GR                   11.50        2.36        0.00       15.66   
Patient R2                    9.45       10.92       11.81        0.00   
Patient R3_2                 10.85        9.24       10.12       11.57   
Patient IT                    9.67        9.43       10.48       11.54   

                  Patient R3_2  Patient IT  
Central Hospital          7.34        9.27  
Patient DT               12.17        9.40  
Patient GR               10.05       12.27  
Patient R2                4.07        7.32  
Patient R3_2              0.00        8.72  
Patient IT                5.93        0.00  


In [15]:
# --- Cell 5: Route Optimization Class ---
class AmbulanceRouter:
    def __init__(self, distance_matrix, hospital_name, max_stops=3):
        self.distance_matrix = distance_matrix
        self.hospital_name = hospital_name
        self.max_stops = max_stops
        self.patient_names = [name for name in distance_matrix.keys() if name != hospital_name]

    def calculate_trip_distance(self, trip):
        """Calculate total distance for a trip (including return to hospital)"""
        if not trip:
            return 0

        # Hospital -> Patient1 -> Patient2 -> ... -> Hospital
        total_distance = self.distance_matrix[self.hospital_name][trip[0]]

        for i in range(len(trip) - 1):
            total_distance += self.distance_matrix[trip[i]][trip[i+1]]

        total_distance += self.distance_matrix[trip[-1]][self.hospital_name]
        return total_distance

    def generate_all_possible_trips(self):
        """Generate all possible valid trips (1-3 patients)"""
        all_trips = []
        for num_stops in range(1, self.max_stops + 1):
            for combo in itertools.combinations(self.patient_names, num_stops):
                # Generate all permutations for each combination
                for perm in itertools.permutations(combo):
                    all_trips.append(list(perm))
        return all_trips

    def brute_force_optimization(self):
        """Brute force optimization to find best route combination"""
        all_trips = self.generate_all_possible_trips()
        best_total_distance = float('inf')
        best_routes = []

        # Try all combinations of trips that cover all patients
        for num_trips in range(1, len(self.patient_names) + 1):
            for trip_combination in itertools.combinations(all_trips, num_trips):
                # Check if this combination covers all patients exactly once
                covered_patients = set()
                for trip in trip_combination:
                    covered_patients.update(trip)

                if covered_patients == set(self.patient_names):
                    total_distance = sum(self.calculate_trip_distance(trip) for trip in trip_combination)
                    if total_distance < best_total_distance:
                        best_total_distance = total_distance
                        best_routes = list(trip_combination)

        return best_routes, best_total_distance


    def optimized_greedy(self):
        """Optimized greedy approach with better heuristics"""
        remaining_patients = set(self.patient_names)
        routes = []
        total_distance = 0

        while remaining_patients:
            # Start from hospital
            current_location = self.hospital_name
            current_trip = []
            current_distance = 0

            # Add up to max_stops patients to this trip
            for _ in range(self.max_stops):
                if not remaining_patients:
                    break

                # Find closest patient to current location
                closest_patient = None
                min_distance = float('inf')

                for patient in remaining_patients:
                    dist = self.distance_matrix[current_location][patient]
                    if dist < min_distance:
                        min_distance = dist
                        closest_patient = patient

                if closest_patient:
                    current_trip.append(closest_patient)
                    current_distance += min_distance
                    current_location = closest_patient
                    remaining_patients.remove(closest_patient)

            # Add return to hospital
            if current_trip:
                current_distance += self.distance_matrix[current_location][self.hospital_name]
                routes.append(current_trip)
                total_distance += current_distance

        return routes, total_distance


In [16]:
# Initialize router
router = AmbulanceRouter(distance_matrix, "Central Hospital")

# Use optimized greedy approach
optimal_routes, total_distance = router.optimized_greedy()

print(f"\nOptimal Routes (Total Distance: {total_distance:.2f} km):")
for i, route in enumerate(optimal_routes, 1):
    print(f"Trip {i}: Hospital -> {' -> '.join(route)} -> Hospital (Distance: {router.calculate_trip_distance(route):.2f} km)")


Optimal Routes (Total Distance: 58.71 km):
Trip 1: Hospital -> Patient DT -> Patient GR -> Patient R3_2 -> Hospital (Distance: 28.46 km)
Trip 2: Hospital -> Patient R2 -> Patient IT -> Hospital (Distance: 30.25 km)


In [17]:
# --- Cell 6: Get OSRM Routes for Visualization ---
def get_osrm_route(coords):
    """Get route geometry from OSRM"""
    coords_str = ';'.join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            return data['routes'][0]['geometry']['coordinates']
    except:
        pass

    return None

def create_interactive_map(locations_df, optimal_routes, hospital_name):
    """Create interactive map with optimal routes"""
    # Create base map centered on hospital
    hospital = locations_df[locations_df['name'] == hospital_name].iloc[0]
    m = folium.Map(location=[hospital['latitude'], hospital['longitude']], zoom_start=13)

    # Add hospital marker
    folium.Marker(
        [hospital['latitude'], hospital['longitude']],
        popup=hospital_name,
        icon=folium.Icon(color='red', icon='hospital-o', prefix='fa')
    ).add_to(m)

    # Add patient markers
    for idx, row in locations_df[locations_df['type'] == 'patient'].iterrows():
        folium.Marker(
            [row['latitude'], row['longitude']],
            popup=row['name'],
            icon=folium.Icon(color='blue', icon='user-md', prefix='fa')
        ).add_to(m)

    # Plot each route with different colors
    colors = ['green', 'purple', 'orange', 'darkred', 'lightblue']

    for i, route in enumerate(optimal_routes):
        if not route:
            continue

        # Get coordinates for the complete route
        route_coords = []

        # Hospital to first patient
        hospital_coord = (hospital['latitude'], hospital['longitude'])
        first_patient = locations_df[locations_df['name'] == route[0]].iloc[0]
        first_coord = (first_patient['latitude'], first_patient['longitude'])

        segment = get_osrm_route([hospital_coord, first_coord])
        if segment:
            route_coords.extend([(lat, lon) for lon, lat in segment])

        # Between patients
        for j in range(len(route) - 1):
            patient1 = locations_df[locations_df['name'] == route[j]].iloc[0]
            patient2 = locations_df[locations_df['name'] == route[j+1]].iloc[0]
            coord1 = (patient1['latitude'], patient1['longitude'])
            coord2 = (patient2['latitude'], patient2['longitude'])

            segment = get_osrm_route([coord1, coord2])
            if segment:
                route_coords.extend([(lat, lon) for lon, lat in segment][1:])  # Skip first point to avoid duplicates

        # Last patient to hospital
        last_patient = locations_df[locations_df['name'] == route[-1]].iloc[0]
        last_coord = (last_patient['latitude'], last_patient['longitude'])

        segment = get_osrm_route([last_coord, hospital_coord])
        if segment:
            route_coords.extend([(lat, lon) for lon, lat in segment][1:])

        # Add route to map
        if route_coords:
            folium.PolyLine(
                route_coords,
                color=colors[i % len(colors)],
                weight=5,
                opacity=0.7,
                popup=f"Trip {i+1}: {' -> '.join(route)}"
            ).add_to(m)

    return m


In [18]:
# Create interactive map
route_map = create_interactive_map(locations_df, optimal_routes, "Central Hospital")
route_map.save('ambulance_routes_real_roads.html')
print("Interactive map with real roads saved as 'ambulance_routes_real_roads.html'")

# Display the map
route_map

Interactive map with real roads saved as 'ambulance_routes_real_roads.html'


In [19]:
# --- Cell 6b: Point-to-Point Best Route Helper Functions ---
def best_route_between_points(lat1, lon1, lat2, lon2, return_geometry=False):
    """
    Calculate the best driving route and distance between two points
    using OSRM (lat/lon coordinates).

    Parameters:
        lat1, lon1 : float - coordinates of point A
        lat2, lon2 : float - coordinates of point B
        return_geometry : bool - if True, also return route geometry

    Returns:
        distance_km : float
        duration_min : float
        geometry : list (only if return_geometry=True)
    """
    coords_str = f"{lon1},{lat1};{lon2},{lat2}"
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            route = data['routes'][0]
            distance_km = route['distance'] / 1000  # meters → km
            duration_min = route['duration'] / 60   # seconds → minutes
            geometry = route['geometry']['coordinates']

            # Debug print
            print(f"Best route from ({lat1}, {lon1}) → ({lat2}, {lon2})")
            print(f"  Distance: {distance_km:.2f} km")
            print(f"  Duration: {duration_min:.2f} min")

            if return_geometry:
                return distance_km, duration_min, geometry
            return distance_km, duration_min
        else:
            print("OSRM error: No route found")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None


def show_route_on_map(lat1, lon1, lat2, lon2):
    """
    Show best route between two points on a folium map.
    """
    result = best_route_between_points(lat1, lon1, lat2, lon2,return_geometry=True)
    if not result:
        return None
    _, _, geometry = result

    # Center map between points
    m = folium.Map(location=[(lat1+lat2)/2, (lon1+lon2)/2], zoom_start=13)
    folium.Marker([lat1, lon1], popup="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker([lat2, lon2], popup="End", icon=folium.Icon(color="red")).add_to(m)

    # Add route polyline
    folium.PolyLine([(lat, lon) for lon, lat in geometry], color="blue", weight=5, opacity=0.7).add_to(m)
    return m


In [20]:
# Coordinates: hospital and Patient DT
# Example: Hospital → Patient DT
lat_h, lon_h = 29.99512653425452, 31.68462840171934
lat_p, lon_p = 30.000417586266437, 31.73960813272627

# Just get distance and duration
dist, dur = best_route_between_points(lat_h, lon_h, lat_p, lon_p)
print("Returned distance:", dist, "km")
print("Returned duration:", dur, "min")

# Get distance, duration + geometry (for drawing route)
dist, dur, geom = best_route_between_points(lat_h, lon_h, lat_p, lon_p, return_geometry=True)
print("Geometry points:", len(geom))

Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min
Returned distance: 8.6285 km
Returned duration: 13.183333333333334 min
Best route from (29.99512653425452, 31.68462840171934) → (30.000417586266437, 31.73960813272627)
  Distance: 8.63 km
  Duration: 13.18 min
Geometry points: 179


In [21]:
# --- Cell 7: Detailed Route Analysis ---
def analyze_routes(locations_df, distance_matrix, optimal_routes, hospital_name):
    """Detailed analysis of the optimal routes"""
    print("\n" + "="*60)
    print("DETAILED ROUTE ANALYSIS")
    print("="*60)

    total_distance = 0
    for i, route in enumerate(optimal_routes, 1):
        trip_distance = 0
        print(f"\nTrip {i}: Hospital -> {' -> '.join(route)} -> Hospital")

        # Hospital to first patient
        dist1 = distance_matrix[hospital_name][route[0]]
        trip_distance += dist1
        print(f"  Hospital → {route[0]}: {dist1:.2f} km")

        # Between patients
        for j in range(len(route) - 1):
            dist = distance_matrix[route[j]][route[j+1]]
            trip_distance += dist
            print(f"  {route[j]} → {route[j+1]}: {dist:.2f} km")

        # Last patient to hospital
        dist2 = distance_matrix[route[-1]][hospital_name]
        trip_distance += dist2
        print(f"  {route[-1]} → Hospital: {dist2:.2f} km")

        print(f"  Total trip distance: {trip_distance:.2f} km")
        total_distance += trip_distance

    print(f"\nOverall total distance: {total_distance:.2f} km")
    print(f"Number of trips: {len(optimal_routes)}")
    print(f"Average patients per trip: {len(locations_df[locations_df['type'] == 'patient']) / len(optimal_routes):.2f}")


In [22]:
# Perform detailed analysis
analyze_routes(locations_df, distance_matrix, optimal_routes, "Central Hospital")


DETAILED ROUTE ANALYSIS

Trip 1: Hospital -> Patient DT -> Patient GR -> Patient R3_2 -> Hospital
  Hospital → Patient DT: 8.63 km
  Patient DT → Patient GR: 2.36 km
  Patient GR → Patient R3_2: 10.12 km
  Patient R3_2 → Hospital: 7.34 km
  Total trip distance: 28.46 km

Trip 2: Hospital -> Patient R2 -> Patient IT -> Hospital
  Hospital → Patient R2: 9.45 km
  Patient R2 → Patient IT: 11.54 km
  Patient IT → Hospital: 9.27 km
  Total trip distance: 30.25 km

Overall total distance: 58.71 km
Number of trips: 2
Average patients per trip: 2.50


In [24]:
# --- Cell 8: Performance Comparison ---
def compare_algorithms():
    """Compare different optimization algorithms"""
    print("\n" + "="*60)
    print("ALGORITHM COMPARISON")
    print("="*60)

    # Greedy optimized
    start_time = time.time()
    routes_greedy, dist_greedy = router.optimized_greedy()
    time_greedy = time.time() - start_time

    print(f"Optimized Greedy:")
    print(f"  Distance: {dist_greedy:.2f} km")
    print(f"  Time: {time_greedy:.4f} seconds")
    print(f"  Trips: {len(routes_greedy)}")



    # Try brute force for small instances
    if len(router.patient_names) <= 5:
        try:
            start_time = time.time()
            routes_brute, dist_brute = router.brute_force_optimization()
            time_brute = time.time() - start_time

            print(f"\nBrute Force (Optimal):")
            print(f"  Distance: {dist_brute:.2f} km")
            print(f"  Time: {time_brute:.4f} seconds")
            print(f"  Trips: {len(routes_brute)}")

            # Show improvement
            improvement = ((dist_greedy - dist_brute) / dist_brute) * 100
            print(f"\nGreedy is {improvement:.2f}% worse than optimal")

        except Exception as e:
            print(f"\nBrute force failed: {e}")

compare_algorithms()


ALGORITHM COMPARISON
Optimized Greedy:
  Distance: 58.71 km
  Time: 0.0000 seconds
  Trips: 2

Brute Force (Optimal):
  Distance: 57.31 km
  Time: 154.4943 seconds
  Trips: 2

Greedy is 2.44% worse than optimal


In [25]:
# --- Cell 9: Export Results ---
def export_results(locations_df, distance_matrix, optimal_routes, total_distance):
    """Export routing results to CSV"""
    results = []

    for i, route in enumerate(optimal_routes, 1):
        trip_distance = router.calculate_trip_distance(route)
        results.append({
            'trip_number': i,
            'route': ' -> '.join(['Hospital'] + route + ['Hospital']),
            'distance_km': round(trip_distance, 2),
            'patients_served': ', '.join(route),
            'number_of_stops': len(route)
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv('ambulance_routing_results.csv', index=False)

    # Export distance matrix
    distance_df = pd.DataFrame(distance_matrix)
    distance_df.to_csv('distance_matrix.csv')

    print("Results exported to 'ambulance_routing_results.csv' and 'distance_matrix.csv'")
    return results_df

# Export results
results_df = export_results(locations_df, distance_matrix, optimal_routes, total_distance)
print("\nExported Results:")
print(results_df)

Results exported to 'ambulance_routing_results.csv' and 'distance_matrix.csv'

Exported Results:
   trip_number                                              route  \
0            1  Hospital -> Patient DT -> Patient GR -> Patien...   
1            2   Hospital -> Patient R2 -> Patient IT -> Hospital   

   distance_km                       patients_served  number_of_stops  
0        28.46  Patient DT, Patient GR, Patient R3_2                3  
1        30.25                Patient R2, Patient IT                2  
